### Sometimes the modeling table didn't record a parameter for some reason. Scripted this to fix a column that didn't get recorded correctly. (in this case, the smoker column)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
from impala.util import as_pandas
import cPickle as pickle
%matplotlib notebook
plt.style.use('ggplot')

In [2]:
from impala.dbapi import connect
conn = connect(host="socbddn01.sharp.com", port=21050)
cur = conn.cursor()
cur.execute("use poc_sandbox")

In [3]:
def fix_smoker_to_table(masterdf, i):
    '''
    input: df with data, includes the extra columns for id
    output: table with column fixed
    '''

    enc = masterdf.encntr_id.iloc[i]
    timeend = masterdf.timeend.iloc[i]
    
    query = '''
    SELECT ce.encntr_id, ce.event_cd 
    ,ce.event_end_dt_tm AS unix_event_end_tm 
    , ce.result_val, ce.result_units_cd
    FROM clinical_event ce 
    JOIN encounter enc ON enc.encntr_id = ce.encntr_id 
    WHERE ce.encntr_id = '{0}' 
    AND ce.event_end_dt_tm < {1} 
    AND ce.result_status_cd NOT IN ('31', '36')
    AND ce.event_class_cd NOT IN ('654645')
    AND ce.valid_until_dt_tm > 4e12
    AND ce.event_cd IN ('75144985') 
    ORDER BY ce.encntr_id, ce.performed_dt_tm;
    '''.format(enc, timeend)

    cur.execute(query)
    df = as_pandas(cur)
    
    if (df[df.event_cd=='75144985']).empty:
        masterdf.ix[i, 'smoker'] = 0
    elif ( (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Heavy tobacco smoker') | 
    (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Light tobacco smoker') |
    (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Current every day smoker') |
    (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Current some day smoker') |
    (df[df.event_cd=='75144985'].result_val.get_values()[0] == 'Smoker, current status unknown')  ):
        masterdf.ix[i, 'smoker'] = 1
    else: 
        masterdf.ix[i, 'smoker'] = 0
        
    return masterdf

In [3]:
filename = 'RRT_modeling_table_13hr_raw.p'
masterdf_rrt = pickle.load(open(filename, 'rb'))

In [5]:
masterdf_rrt.describe().T

,count,mean,std,min,25%,50%,75%,max
event_end_dt_tm,2063.0,1.445053e+12,1.478512e+10,1.410347e+12,1.431889e+12,1.445573e+12,1.457776e+12,1.471205e+12
timestart,2063.0,1.445006e+12,1.478512e+10,1.410300e+12,1.431842e+12,1.445526e+12,1.457729e+12,1.471158e+12
timeend,2063.0,1.445050e+12,1.478512e+10,1.410343e+12,1.431885e+12,1.445570e+12,1.457772e+12,1.471201e+12
age,2063.0,6.741154e+01,1.660477e+01,1.800000e+01,5.800000e+01,6.900000e+01,8.000000e+01,1.010000e+02
obese,1521.0,2.971729e-01,4.571636e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
smoker,2063.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
prev_rrt,2063.0,1.536597e-01,3.607097e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
on_iv,2063.0,7.280659e-01,4.450640e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
bu-nal,2063.0,3.005332e-02,1.707755e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
DBP_mean,1950.0,7.063889e+01,1.165375e+01,3.791667e+01,6.200000e+01,7.000000e+01,7.866667e+01,1.223125e+02


In [ ]:
count = 0
for i in xrange(len(masterdf_rrt)):
# for i in xrange(27, len(masterdf_rrt), 1):
# for i in range(3):
    count +=1
    print "count {0} of {1}".format(count, len(masterdf_rrt))
    masterdf_rrt = fix_smoker_to_table(masterdf_rrt, i)
masterdf_rrt.to_pickle(filename[:-2] + '_rev1.p')

count 1 of 2063
count 2 of 2063
count 3 of 2063
count 4 of 2063


In [20]:
masterdf_rrt.head()

,rrt_ce_id,encntr_id,event_end_dt_tm,timestart,timeend,age,sex,obese,smoker,prev_rrt,...,GCS_mean,GCS_recent,anticoagulants,narcotics,narc-ans,antipsychotics,chemo,dialysis,race,rrt_reason
0,6758153566,100009623,1441447620000,1441400820000,1441444020000,67,M,NaN,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,White,Arrythmias
1,6802898656,100022347,1442595780000,1442548980000,1442592180000,68,F,0.0,1.0,0.0,...,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,White,"Staff concerned/Worried about Patient, Tachyca..."
2,6820399231,100022347,1443022200000,1442975400000,1443018600000,68,F,0.0,1.0,1.0,...,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,White,"Hypotension, Tachycardia"
3,6764808869,100031243,1441621320000,1441574520000,1441617720000,81,M,0.0,0.0,0.0,...,15.0,15.0,0.0,1.0,0.0,0.0,0.0,0.0,White,Suspected Stroke/Acute
4,6761972438,100031586,1441548900000,1441502100000,1441545300000,68,F,1.0,0.0,0.0,...,12.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,Other Race,Changes in LOC
